In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import time, os
import csv

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [5]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [6]:
def find_page_name(soup):
    try: 
        name = soup.find('h1', id='HEADING').text
    except:
        name = 'empty'
    return name 

In [7]:
def find_rating(soup):
    try:
        rating = soup.find('span', class_="bvcwU P").text
    except: 
        rating = 'empty'
    return rating

In [8]:
def find_address(soup):
    try:
        address = soup.find('span', class_="ceIOZ yYjkv").text
    except:
        address = 'empty'
    return address

In [9]:
def find_amenities(soup):
    try: 
        amenity_tags = soup.find_all(class_="exmBD K")
    except:
        property_amenities = 'empty'
        room_features = 'empty'
        room_types = 'empty'
    else:
        try:
            property_amenities =  [item.text for item in amenity_tags[0].find_all(attrs = {'class':"bUmsU f ME H3 _c", 'data-test-target': 'amenity_text'})]
        except:
            property_amenities = 'empty'
        try: 
            room_features = [item.text for item in amenity_tags[1].find_all(attrs = {'class':"bUmsU f ME H3 _c", 'data-test-target': 'amenity_text'})]
        except:
            room_features = 'empty'
        try:
            room_types = [item.text for item in amenity_tags[2].find_all(attrs = {'class':"bUmsU f ME H3 _c", 'data-test-target': 'amenity_text'})]
        except:
            room_types = 'empty'
    return property_amenities, room_features, room_types
        

In [10]:

#UPDATED 
def find_languages(soup):
    try:
        languages = soup.find(text='Languages Spoken').next.text
    except:
        languages = 'empty'
    return languages
        

In [11]:
def find_hotel_styles(soup):
    hotel_styles = []
    try:
        for item in soup.find_all(class_="dyJVb Ci S2 H2 b Wf"):
            if item.text == 'HOTEL STYLE':
                next_siblings = item.find_next_siblings(class_="drcGn _R MC S4 _a H")
                for sibling in next_siblings:
                    hotel_styles.append(sibling.text)
    except:
        hotel_styles = "empty"
    return hotel_styles 
                                          

In [12]:
def find_star_class(soup):
    try:
        stars = soup.find('svg',class_= 'TkRkB d H0')['title']
    except:
        stars = 'empty'
    return stars
    

In [13]:
def find_walkability(soup):
    try:
        score = soup.find('div', class_="eaCqs u v ui_column is-4").text
    except:
        score = 'empty'
    return score

In [14]:
def find_proximity(soup):
    try:
        distances = [item.text for item in soup.find_all(class_="ehKIl")]
    except:
        restaurants_distance = 'empty'
        attractions_distance = 'empty'
    else:
        try:
            restaurants_distance = soup.find('span', class_ = "bpwqy VyMdE").text + ' ' + soup.find('span', class_ = "hqMZC H3 b bBSTv").text + ' ' + distances[1]
        except:
            restaurants_distance = 'empty'
        try: 
            attractions_distance = soup.find('span', class_ = "bpwqy eKwbS").text + ' ' + soup.find('span', class_ = "hqMZC H3 b fBLNb").text + ' ' + distances[2]
        except:
            attractions_distance = 'empty'
    return restaurants_distance, attractions_distance 

    

In [15]:
def find_reviews(soup):
    try: 
        reviews = soup.find(class_='HFUqL').text
    except:
        reviews = 'empty'
    return reviews

In [16]:
def find_precautions(soup):
    try:
        precautions = [item.text for item in soup.find_all('span', class_="ctpJG")]
    except:
        precautions = 'empty'
    return precautions

In [17]:
def extract_page_elements(hotel_dict):
    try:
        driver.find_element_by_xpath('//*[@id="COVID19"]/div/div[2]/div[1]/div[1]/div[2]').click()
    except:
        pass
    finally:
        soup = BeautifulSoup(driver.page_source, "html.parser")
    
        hotel_dict["Page Name"] = find_page_name(soup) #double checking
        hotel_dict['Tripadvisor Rating'] = find_rating(soup)
        hotel_dict['Address'] = find_address(soup)
    
        amenities = find_amenities(soup)
        hotel_dict['Property Amenities'] = amenities[0]
        hotel_dict['Room Features'] = amenities[1]
        hotel_dict['Room Types'] = amenities[2]
    
        
        hotel_dict['Hotel Style'] = find_hotel_styles(soup)
        hotel_dict['Languages'] = find_languages(soup)
    
        hotel_dict['Hotel Star Class'] = find_star_class(soup)
        hotel_dict['Car/Walkability'] = find_walkability(soup)
    
        nearby = find_proximity(soup)
        hotel_dict['Restaurants Proximity'] = nearby[0]
        hotel_dict['Attractions Proximity'] = nearby[1]
    
        hotel_dict['Number of Reviews'] = find_reviews(soup)
        hotel_dict['Covid Precautions'] = find_precautions(soup)
    return hotel_dict
    
    

In [18]:
URL = "https://www.tripadvisor.com/Hotels-g529023-Shoalhaven_New_South_Wales-Hotels.html"

#driver = webdriver.Chrome(chromedriver,options=options)
driver = webdriver.Chrome(chromedriver)
driver.get(URL)

In [224]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
# default number of scraped pages
num_page = 10

In [ ]:
# Open the file to save the review
#csvFile = open(path_to_file, 'a', encoding="utf-8")
#csvWriter = csv.writer(csvFile)

In [19]:
def making_list(urls, hotels):
    try:
        driver.find_element_by_xpath("//*[@id='HEADING']/span").click()
    except:
        pass
    finally:
        time.sleep(3)
        for num, url in enumerate(urls):
            hotel_dict = {}
            hotel_dict['Name'] = driver.find_elements_by_class_name("listing_title")[num].text
            try:
                hotel_dict['Featured Price'] = driver.find_elements_by_class_name("price-wrap")[num].text
            except:
                hotel_dict['Featured Price'] = 'empty'
            try:
                hotel_dict['Category'] = driver.find_elements_by_class_name("mb10")[num].text
            except:
                hotel_dict['Category'] = 'empty'
            finally:
                main_page = driver.current_window_handle
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//div[@class='listing_title']/a")))
                urls[num].click()  
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(5)
                full_dict = extract_page_elements(hotel_dict)
                hotels.append(full_dict)
                driver.close()
                driver.switch_to.window(main_page)
                time.sleep(3 + 2*random.random())
        return hotels

In [24]:
destinations = {
    #'Phillip Island': 'https://www.tripadvisor.com/Hotels-g504338-Phillip_Island_Victoria-Hotels.html',
    #'Gold Coast': 'https://www.tripadvisor.com/Hotels-g255337-Gold_Coast_Queensland-Hotels.html',
    #'Port Stephens': 'https://www.tripadvisor.com/Hotels-g529017-Port_Stephens_Greater_Newcastle_New_South_Wales-Hotels.html',
    #'Cowes': 'https://www.tripadvisor.com/Hotels-g261657-Cowes_Phillip_Island_Victoria-Hotels.html',
    #'Melbourne': 'https://www.tripadvisor.com/Hotels-g255100-oa210-Melbourne_Victoria-Hotels.html', #broke
    #'Noosa': 'https://www.tripadvisor.com/Hotels-g261678-Noosa_Sunshine_Coast_Queensland-Hotels.html',
    #'Port Douglas': 'https://www.tripadvisor.com/Hotels-g255070-Port_Douglas_Queensland-Hotels.html',
    #'Sydney': 'https://www.tripadvisor.com/Hotels-g255060-Sydney_New_South_Wales-Hotels.html',
    #'Jervis Bay': 'https://www.tripadvisor.com/Hotels-g528972-Jervis_Bay_Shoalhaven_New_South_Wales-Hotels.html',
    #'Kangaroo Island': 'https://www.tripadvisor.com/Hotels-g255095-Kangaroo_Island_South_Australia-Hotels.html',
    #'Airlie Beach': 'https://www.tripadvisor.com/Hotels-g261596-Airlie_Beach_Queensland-Hotels.html', 
    #'Hervey Bay': 'https://www.tripadvisor.com/Hotels-g255404-Hervey_Bay_Fraser_Coast_Queensland-Hotels.html',
    #'Busselton': 'https://www.tripadvisor.com/Hotels-g261672-Busselton_Margaret_River_Region_Western_Australia-Hotels.html',
    #'Toowoomba': 'https://www.tripadvisor.com/Hotels-g255340-Toowoomba_Queensland-Hotels.html'
    }

#doesn't include shoalhaven which was already extracted
#jervis bay contains some shoalhaven locations--remove duplicates during data cleaning oricess

In [21]:
def add_to_csv(list_dict, destination):
    keys = list_dict[0].keys()
    with open(f'{destination}.csv', 'a', newline='') as file:
        dict_writer = csv.DictWriter(file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_dict)
        file.close()

In [25]:
def hotel_scraper(destination):
    while True:
        time.sleep(5)
        #WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT('Next'))))
        next_button = driver.find_elements_by_link_text('Next')
        if len(next_button) <1:
            print("no more pages left")
            break
        else:
            hotels = []
            #driver.switch_to.window(driver.current_window_handle)
            hotel_urls = driver.find_elements_by_xpath(".//div[@class='listing_title']/a")
            #print(hotel_urls)
            list_dict = making_list(hotel_urls, hotels)
            add_to_csv(list_dict, destination)
            driver.find_element_by_link_text('Next').click()
            print('clicked')
            time.sleep(10 + 2*random.random()) 
    driver.close()

In [26]:
 for destination in destinations:
        driver = webdriver.Chrome(chromedriver)
        driver.get(destinations[destination])
        soup = BeautifulSoup(driver.page_source, "html.parser")
        time.sleep(7)
        t0 = time.time()
        hotel_scraper(destination)
        print(destination, time.time() - t0)

clicked
clicked
clicked
no more pages left
Busselton 1238.708848953247
clicked
clicked
no more pages left
Toowoomba 627.0990788936615
